In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
!pip install transformers

In [3]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [4]:
import logging
from huggingface_hub import notebook_login
from pathlib import Path

logging.disable(logging.WARNING)

torch.manual_seed(1)
if not (Path.home()/'.cache/huggingface'/'token').exists(): notebook_login()

## Waiting for approval to use meta Llama

In [5]:
#mn = "meta-llama/Llama-2-7b-hf"

In [6]:
# waiting for approval for meta-Llma from meta repo on Huggingface
#model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_8bit=True)

In [8]:
!pip install accelerate

In [9]:
!pip install auto-gptq
!pip install --upgrade optimum

In [10]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)


## LLM from stabilityAi
I can not load this model on this nb since i am using Google Colab free version the Disk size can not handle the model it since it fill the 78.2 GB allocated storage.
So be advised if you are experimenting with Colab

The GPTQ optimized version can still work using colab (N.B. you might need to Restart your nb after installing the required libraries above, but it works.

In [11]:
#model = AutoModelForCausalLM.from_pretrained("stabilityai/StableBeluga2", torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto")
#system_prompt = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can. Remember, be safe, and don't do anything illegal.\n\n"


In [12]:
MN_= "TheBloke/Llama-2-7b-Chat-GPTQ"
#mn = "stabilityai/StableBeluga2"

In [13]:
tokr = AutoTokenizer.from_pretrained(MN_)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

In [14]:
toks

{'input_ids': tensor([[    1,  5677,  6764, 17430,   338,   263, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [15]:
tokr.batch_decode(toks['input_ids'])

['<s> Jeremy Howard is a ']

In [20]:
%%time

res = model.generate(**toks.to("cuda"), max_new_tokens=300).to("cpu")
res

CPU times: user 15.3 s, sys: 81 µs, total: 15.3 s
Wall time: 17.9 s


tensor([[    1,  5677,  6764, 17430,   338,   263, 29871, 29941, 29945, 29899,
          6360, 29899,  1025,   767,   515,   278,  3303,  3900,  1058,   471,
         24383,   297,   498, 26517,   363, 16831, 23244, 15661,   304,  1560,
           688,  6234, 29871, 29896, 29889, 29941,  4679,   468, 25402,   310,
         10901, 11195,   286,   621,   314,   561,   300,   314,   457,   964,
           278,  4234, 29889,    13,  7504,  3278,   304,   263,  3461,   491,
           278, 14320, 29895,   554,  4918, 29892, 17430,   471, 24383,   472,
          2166,  1707,  7183, 29882, 15547, 18117,   297, 14320, 29895,   554,
          1156,  2888, 29879, 24921, 10943,   278, 15721,  7934,   297,   263,
          3577,   310, 11966,  1199,   540,   471, 19436, 29889,    13,  5328,
           538,  8967,   368, 20186,   304, 15661,   304,  1560,   688,  6234,
           278, 15721,   964,   498, 26517, 29892,   322,   540,   756,  1063,
         20139,   411, 15721,  1020,   600,   860,  

In [21]:
tokr.batch_decode(res)

["<s> Jeremy Howard is a 35-year-old man from the United States who was arrested in Thailand for allegedly attempting to smuggle 1.3 kilograms of crystal methamphetamine into the country.\nAccording to a report by the Bangkok Post, Howard was arrested at Suvarnabhumi Airport in Bangkok after customs officials discovered the drug hidden in a package of electronics he was carrying.\nHoward reportedly admitted to attempting to smuggle the drug into Thailand, and he has been charged with drug trafficking. If convicted, he could face up to 15 years in prison.\nIt is not uncommon for foreigners to be arrested in Thailand for drug-related offenses, as the country has strict laws and penalties for drug trafficking. However, in this case, Howard's arrest may have been facilitated by his use of a local airline to transport the drug.\nThai Airways International, the national carrier of Thailand, has a strict policy of not allowing passengers to carry any prohibited items, including drugs, on thei